In [107]:
#!pip install surprise


In [122]:
from surprise import Dataset, Reader, SVD
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
import pandas as pd

In [109]:
#Leemos los datos de las reseñas / Ejemplo con Reviews_9.csv
df = pd.read_csv('reviews_9.csv', sep=';', escapechar='\\')

In [110]:
#df.head()

In [111]:
#Crea un objeto Reader y Dataset de Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[["Id_Usuario", "Id_Restaurant", "Rating"]], reader)

In [112]:
#Divide los datos en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

In [113]:
#Instanciamos y entrenamos el modelo SVD
algo = SVD()
algo.fit(trainset)

In [114]:
#Hacemos una predicción
predictions = algo.test(testset)

In [115]:
#Obtenemos las reseñas del usuario
user_reviews = {}
for user_id, restaurant_id, rating in df[["Id_Usuario", "Id_Restaurant", "Rating"]].values:
  if user_id not in user_reviews:
    user_reviews[user_id] = []
  user_reviews[user_id].append((restaurant_id, rating))

In [116]:
#Pide al usuario que ingrese su ID de usuario
user_id = input("Ingrese el Id de su usuario: ")

Ingrese el Id de su usuario: z0Yh6ntpynyTP6x0YLCnLA


In [117]:
#Hacemos la recomendacion de los restaurantes que el usuario aún no ha visitado
restaurant_recs = []
for restaurant_id in df["Id_Restaurant"].unique():
  if restaurant_id not in [r_iid for r_iid, _ in user_reviews.get(user_id, [])]:
    predicted_rating = algo.predict(user_id, restaurant_id).est
    restaurant_recs.append((restaurant_id, predicted_rating))

In [118]:
#Ordena las recomendaciones por calificación
restaurant_recs = sorted(restaurant_recs, key=lambda x: x[1], reverse=True)

In [119]:
#Muestra las 5 mejores recomendaciones al usuario
print("Recomendaciones para el usuario: {}".format(user_id))
for restaurant_id, predicted_rating in restaurant_recs[:5]:
  print("- Restaurante {}: Prediccion de puntaje {:.2f}".format(restaurant_id, predicted_rating))

Recomendaciones para el usuario: z0Yh6ntpynyTP6x0YLCnLA
- Restaurante QoezRbYQncpRqyrLH6Iqjg: Prediccion de puntaje 4.57
- Restaurante 9xdXS7jtWjCVzL4_oPGv9A: Prediccion de puntaje 4.56
- Restaurante lqF0Q5vVLxY-_Cth6LtEdg: Prediccion de puntaje 4.49
- Restaurante ytynqOUb3hjKeJfRj5Tshw: Prediccion de puntaje 4.47
- Restaurante K7KHmHzxNwzqiijSJeKe_A: Prediccion de puntaje 4.46


#Evaluación del modelo

In [120]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

In [123]:
#Evaluamos con RMSE (error cuadrático medio)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.3644


1.3644194321204535